In [1]:
from qiskit import *
from qiskit.circuit.library import DraperQFTAdder, CXGate, CCXGate, MCXGate
from qiskit.quantum_info import Statevector, partial_trace
from qiskit import Aer
from qiskit_ibm_provider import IBMProvider
from qiskit.visualization import plot_histogram

import time

In [2]:
# carry gate 

# |0> - previous carry
# |1> - a bit (addend bit)
# |2> - b bit (addend bit)
# |3> - c bit next carry bit

carry_circ = QuantumCircuit(4, name='carry')
carry_circ.ccx(1, 2, 3)
carry_circ.cx(1, 2)
carry_circ.ccx(0, 2, 3)

carry = carry_circ.to_gate()

carry_inv = carry.inverse()

carry_circ.draw()

q_0: ────────────■──
                 │  
q_1: ──■────■────┼──
       │  ┌─┴─┐  │  
q_2: ──■──┤ X ├──■──
     ┌─┴─┐└───┘┌─┴─┐
q_3: ┤ X ├─────┤ X ├
     └───┘     └───┘

In [3]:
# sum gate
sum_circ = QuantumCircuit(3, name='sum')
sum_circ.cx(1, 2)
sum_circ.cx(0, 2)

sum_gate = sum_circ.to_gate()

sum_circ.draw()

q_0: ───────■──
            │  
q_1: ──■────┼──
     ┌─┴─┐┌─┴─┐
q_2: ┤ X ├┤ X ├
     └───┘└───┘

In [6]:
# plain adder circuit
def add(a, b):

    if (a > b):
        num_qubits = len(str(bin(a))) - 2
    else:
        num_qubits = len(str(bin(b))) - 2
        
    if num_qubits > 9:
        raise Exception('Qubit limit of simulator exceeded, each addend should be less than 511')
    
    # creating the circuits
    qr_a = QuantumRegister(num_qubits, name='a')
    qr_b = QuantumRegister(num_qubits + 1, name='b')
    qr_c = QuantumRegister(num_qubits, name='c') # Carry qubits
    cl = ClassicalRegister(num_qubits + 1, name='cl') # for measurements
    adder = QuantumCircuit(qr_a, qr_b, qr_c, cl)
    
    # initializing the state with the input values
    state_a = Statevector.from_int(a, 2**num_qubits)
    state_b = Statevector.from_int(b, 2**(num_qubits+1))
    state_c = Statevector.from_int(0, 2**num_qubits)
    adder.initialize(state_a, range(num_qubits)) # 0, 1, 2
    adder.initialize(state_b, range(num_qubits, 2*num_qubits+1)) # 3, 4, 5, 6
    adder.initialize(state_c, range(2*num_qubits+1, 3*num_qubits+1)) # 7, 8, 9
    
    # adding carry gates
    for i in range(num_qubits - 1):
        adder.append(carry, [qr_c[i], qr_a[i], qr_b[i], qr_c[i+1]])

    # for last iteration carry gate stores things differently
    adder.append(carry, [qr_c[num_qubits-1], qr_a[num_qubits-1], qr_b[num_qubits-1], qr_b[num_qubits]])

    # reversing whole stuff up

    # last iteration again
    adder.cx(qr_c[num_qubits-1], qr_b[num_qubits-1])

    # rest of them
    for i in range(num_qubits - 1):
        adder.append(carry_inv, [qr_c[(num_qubits-2)-i], qr_a[(num_qubits-2)-i], qr_b[(num_qubits-2)-i], qr_c[(num_qubits-1)-i]])
        adder.append(sum_gate, [qr_a[(num_qubits-2)-i], qr_c[(num_qubits-2)-i], qr_b[(num_qubits-2)-i]])

    adder.barrier() # necessary to separate out mesaurement gates together
    
    # measurement into classical qubits
    for i in range(num_qubits + 1):
        adder.measure(qr_b[i], cl[i])
    
    # simulating for results - using qasm simulator
    simulator = Aer.get_backend('aer_simulator')
    job = simulator.run(transpile(adder, backend=simulator))
    counts = job.result().get_counts(adder)

#     # ibm cloud simulator - gives transpile errors
#     provider = IBMProvider()
#     backend = provider.get_backend('ibmq_qasm_simulator')
#     job = backend.run(transpile(adder, backend=backend))
#     counts = job.result().get_counts(adder)
    
    
    return int(list(counts.keys())[0], 2)

In [7]:
start1 = time.time()

res1 = add(5, 4)

timing1 = time.time() - start1

print(res1, f'time taken: {timing1}')

9 time taken: 2.538285732269287


In [ ]:
start2 = time.time()

res2 = add(45, 40)

timing2 = time.time() - start2

print(res2, f'time taken: {timing2}')

In [ ]:
start3 = time.time()

res3 = add(4, 100)

timing3 = time.time() - start3

print(res3, f'time taken: {timing3}')

In [ ]:
start4 = time.time()

res4 = add(200, 400)

timing4 = time.time() - start4

print(res4, f'time taken: {timing4}')

In [ ]:
print(add(-5, 15)) # wrong answer

### Fixing transpile error --- ignore this section

In [16]:
# telling transpiler not to decompose
qc_transed = transpile(custom_circ, basis_gates=basis_gates)
qc_transed.draw()

TranspilerError: "Unable to translate the operations in the circuit: ['cx', 'carry'] to the backend's (or manually specified) target basis: ['reset', 'delay', 'carry', 'CXGate', 'barrier', 'snapshot', 'measure', 'sum_gate']. This likely means the target basis is not universal or there are additional equivalence rules needed in the EquivalenceLibrary being used. For more details on this error see: https://qiskit.org/documentation/stubs/qiskit.transpiler.passes.BasisTranslator.html#translation_errors"

In [15]:
from qiskit.compiler import transpile
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary 

basis_gates = ['CXGate', 'carry', 'sum_gate']

custom_circ = QuantumCircuit(4)
custom_circ.append(sum_gate, range(1,4))
custom_circ.append(carry, range(4))
custom_circ.append(CXGate(), (0, 3))

custom_circ.draw()

┌────────┐     
q_0: ────────┤0       ├──■──
     ┌──────┐│        │  │  
q_1: ┤0     ├┤1       ├──┼──
     │      ││  carry │  │  
q_2: ┤1 sum ├┤2       ├──┼──
     │      ││        │┌─┴─┐
q_3: ┤2     ├┤3       ├┤ X ├
     └──────┘└────────┘└───┘

In [19]:
sum_def = QuantumCircuit(3)
sum_def.append(sum_gate, range(3))
sum_def.draw()
SessionEquivalenceLibrary.add_equivalence(CCXGate(), sum_def)

carry_def = QuantumCircuit(4)
carry_def.append(carry, range(4))
carry_def.draw()
SessionEquivalenceLibrary.add_equivalence(CCXGate(), carry)

CircuitError: 'Cannot add equivalence between circuit and gate of different shapes. Gate: 3 qubits and 0 clbits. Circuit: 4 qubits and 0 clbits.'